In [1]:
# !pip install ee
# !pip install geemap

In [2]:
import ee
import geemap

In [3]:
# Initialize both geemap and Google Earth Engine
Map = geemap.Map()
ee.Initialize()

# Get polygon of Montreal to clip future datasets to
GAUL = ee.FeatureCollection("FAO/GAUL/2015/level2")
montreal = GAUL.filter(ee.Filter.eq('ADM2_NAME', 'Montréal'))

# Get the Landsat 9 Imagery
landsat = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# use Median on the ImageCollection to view a generalized image
median = landsat.median()

In [4]:
# Need to rescale since they compress the data to save memory, to compute we need to rescale

def applyScaleFactors(image: ee.Image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)
    

In [5]:
data = applyScaleFactors(median)
dataset = data.clipToCollection(montreal)

In [6]:
# Landsat 9 natural images visualization parameters
vis_natural = {
    # Select the bands to display
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.4,
}

# Landsat 9 NIR images visualization parameters
vis_nir = {
    # Select the bands to display
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0.0,
    'max': 0.5,
}

In [7]:

left_layer = geemap.ee_tile_layer(dataset, vis_natural, 'Landsat9 Natural (432)')
right_layer = geemap.ee_tile_layer(dataset, vis_nir, 'Landsat9 NIR (543)')

In [11]:
Map.split_map(left_layer = left_layer,
              right_layer = right_layer,
              left_label = 'Landsat 9 Natural Color (432)',
              right_label = 'Landsat 9 Infrared Color (543)',
            )


In [14]:
Map

Map(bottom=47334.0, center=[45.22461173085722, -72.54684448242189], controls=(ZoomControl(options=['position',…

In [13]:
geemap.Report()

--------------------------------------------------------------------------------
  Date: Sun Apr 02 17:07:52 2023 Eastern Daylight Time

                OS : Windows
            CPU(s) : 8
           Machine : AMD64
      Architecture : 64bit
               RAM : 16.0 GiB
       Environment : Jupyter

  Python 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]

            geemap : 0.20.4
                ee : 0.1.347
        ipyleaflet : 0.17.2
            folium : 0.14.0
        jupyterlab : 3.4.4
          notebook : 6.4.12
         ipyevents : 2.0.1
         geopandas : 0.12.2

  Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904
  for Intel(R) 64 architecture applications
--------------------------------------------------------------------------------